In [1]:
import pandas as pd
import numpy as np

import ibm_db
import ibm_db_dbi
import ibm_db_sa

import datetime as dt
import time
from datetime import date
from dateutil import relativedelta

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import os
from tqdm import tqdm_notebook
import sqlalchemy
from functools import reduce
import pickle
import joblib
import io

import os
from project_lib import Project
project = Project()
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()

In [2]:
dsn = "DRIVER={IBM DB2 ODBC DRIVER};DATABASE=bludb;HOSTNAME=10.1.50.25;PORT=50000;PROTOCOL=TCPIP;UID=tpbdatait1;PWD=Tpbdatait1@123;"
hdbc = ibm_db.connect(dsn, "", "")
hdbi = ibm_db_dbi.Connection(hdbc)

pd.options.display.float_format = '{:.2f}'.format
np.set_printoptions(suppress= True)

In [3]:
QUERY_EB_ALL_TRANS = """
        SELECT SO_CIF,
            DATE('{obs_date}') as OBS_DATE,
            COUNT(SO_TIEN) AS EB_NUM_TRANS_OUT_{lookback_days},
            SUM(SO_TIEN) AS EB_SUM_TRANS_AMT_OUT_{lookback_days},
            AVG(SO_TIEN) AS EB_AVG_TRANS_AMT_OUT_{lookback_days},
            STDDEV_SAMP(SO_TIEN) AS EB_STD_TRANS_AMT_OUT_{lookback_days},
            MIN(SO_TIEN) AS EB_MIN_TRANS_AMT_OUT_{lookback_days},
           MAX(SO_TIEN) AS EB_MAX_TRANS_AMT_OUT_{lookback_days}
           
        FROM   CSO.HBK_MIS064D_TRANS_INFO e
        
        WHERE  SO_TIEN >=0                              -- loại bỏ các giao dịch có giá trị <= 0 do có thể là GD hoàn tiền hoặc bị lỗi ghi nhận dữ liệu
            AND    SO_CIF = LEFT(TK_CHUYEN, 8)          -- chỉ lấy các giao dịch đầu ra 
            AND loai_gd not like '%STK%'
            AND e.TRN_DT <= DATE('{obs_date}')          -- các giao dịch phải trong khoảng từ ngày quan sát trừ đi số ngày look back
            AND e.TRN_DT > DATE('{obs_date}')  - {lookback_days}
        GROUP  BY SO_CIF, DATE('{obs_date}') 
        """

QUERY_CASA_CREDIT_TRANS = """
        SELECT LEFT(AC_NO, 8) AS SO_CIF, 
           DATE('{obs_date}') AS OBS_DATE,
           SUM(AMT_LCY) AS CASA_ACC_SUM_TRANS_AMT_IN_{lookback_days},
           COUNT(TRN_REF_NO) AS CASA_ACC_NUM_TRANS_AMT_IN_{lookback_days},
           AVG(AMT_LCY) AS CASA_ACC_AVG_TRANS_AMT_IN_{lookback_days},
           STDDEV_SAMP(AMT_LCY) AS CASA_ACC_STD_TRANS_AMT_IN_{lookback_days},
           MIN(AMT_LCY) AS CASA_ACC_MIN_TRANS_AMT_IN_{lookback_days},
           MAX(AMT_LCY) AS CASA_ACC_MAX_TRANS_AMT_IN_{lookback_days}
       
        FROM CSO.TTDATA_ST_TXN e

        WHERE 
            TXN_DRC_F = 'C' 
            AND AMT_LCY > 0
            AND PPN_DT <= DATE('{obs_date}')
            AND PPN_DT > DATE('{obs_date}') - {lookback_days}
            -- AND TXN_TP = 'S'
        GROUP BY LEFT(AC_NO, 8), DATE('{obs_date}')
        """

QUERY_CASA_DEBIT_TRANS = """
        SELECT LEFT(AC_NO, 8) AS SO_CIF, 
           DATE('{obs_date}') AS OBS_DATE,
           SUM(AMT_LCY) AS CASA_ACC_SUM_TRANS_AMT_OUT_{lookback_days},
           COUNT(TRN_REF_NO) AS CASA_ACC_NUM_TRANS_AMT_OUT_{lookback_days},
           AVG(AMT_LCY) AS CASA_ACC_AVG_TRANS_AMT_OUT_{lookback_days},
           STDDEV_SAMP(AMT_LCY) AS CASA_ACC_STD_TRANS_AMT_OUT_{lookback_days},
           MIN(AMT_LCY) AS CASA_ACC_MIN_TRANS_AMT_OUT_{lookback_days},
           MAX(AMT_LCY) AS CASA_ACC_MAX_TRANS_AMT_OUT_{lookback_days}
        
        FROM CSO.TTDATA_ST_TXN e

        WHERE 
            TXN_DRC_F = 'D'
            AND AMT_LCY > 0
            -- AND TXN_TP = 'S'
            AND PPN_DT <= DATE('{obs_date}')
            AND PPN_DT > DATE('{obs_date}') - {lookback_days}
        GROUP BY LEFT(AC_NO, 8), DATE('{obs_date}')
        """

QUERY_NO_ACCT_OUT = """
        SELECT 
            LEFT(tk_chuyen, 8) AS SO_CIF,
            DATE('{obs_date}') AS OBS_DATE,
            count(DISTINCT TK_NHAN) AS eb_tk_nhan_count_{lookback_days}
        FROM cso.HBK_MIS064D_TRANS_INFO hmdti
        WHERE 
            year(TRN_DT) = '2022'
            AND SO_CIF = left(TK_CHUYEN, 8)
            AND TRN_DT <= DATE('{obs_date}')                          
            AND TRN_DT > DATE('{obs_date}') - {lookback_days} 
        GROUP BY LEFT(TK_CHUYEN,8), DATE('{obs_date}')
        """

In [4]:
def get_feature(l_obs_date, l_lookback_days, query, name):
    dict_df_feature = {}
    
    for obs_date in l_obs_date:
        for lookback_days in l_lookback_days:
            df = pd.read_sql_query(query.format(obs_date=obs_date, lookback_days=lookback_days), hdbi)
            dict_df_feature[obs_date + '_' + name + '_' + str(lookback_days)] = df
    return dict_df_feature

def group_feature_daybyday(l_obs_date, l_lookback_days, query, name):
    """ NHOM CAC FEATURE THEO TUNG NGAY LOOKBACK"""
    dict_df_feature = get_feature(l_obs_date, l_lookback_days, query, name)
    
    df_28_days, df_56_days, df_84_days = [pd.DataFrame() for x in range(3)]
    for key in dict_df_feature.keys():
        if key[-2:] == '28':
            lookback_28_days = dict_df_feature[key]
            df_28_days = df_28_days.append(lookback_28_days)
        if key[-2:] == '56':
            lookback_56_days = dict_df_feature[key]
            df_56_days = df_56_days.append(lookback_56_days)
        if key[-2:] == '84':
            lookback_84_days = dict_df_feature[key]
            df_84_days = df_84_days.append(lookback_84_days)
    df_query = df_28_days.merge(df_56_days, on = ['SO_CIF', 'OBS_DATE'], how='outer').merge(df_84_days, on = ['SO_CIF', 'OBS_DATE'], how='outer')
    return df_query

def concat_feature(dict_df_feature, l_obs_date, l_lookback_days):
    """ CONCAT CAC NHOM FEATURE KHAC NHAU"""
    df_concat_ft = pd.DataFrame(columns={'SO_CIF','OBS_DATE'})
    for key, value in tqdm_notebook(dict_df_feature.items()):
        df_query = group_feature_daybyday(l_obs_date, l_lookback_days, value, key)
        df_concat_ft = df_concat_ft.merge(df_query, left_on=['SO_CIF', 'OBS_DATE'], right_on=['SO_CIF', 'OBS_DATE'], how='outer')
    return df_concat_ft

def create_feature(label_file, dict_df_feature):
    buffer = wslib.load_data(label_file)
    df_label = pickle.load(buffer)
    df_label.obs_date = df_label.obs_date.astype(str)
    
    l_obs_date = list(df_label.obs_date.astype(str).unique())
    l_lookback_days = [28*n for n in range(1,4)]
    
    df_concat_ft = concat_feature(dict_df_feature, l_obs_date, l_lookback_days)
    df_concat_ft.OBS_DATE = df_concat_ft.OBS_DATE.astype(str)
    
    df_feature = df_label.merge(df_concat_ft, left_on=['cif_number', 'obs_date'], right_on=['SO_CIF', 'OBS_DATE'], how='left')
    df_feature.drop(columns=['SO_CIF', 'OBS_DATE'], inplace=True)
    return df_feature

In [5]:
dict_df_feature = {'eb_trans_out':QUERY_EB_ALL_TRANS
                   , 'casa_trans_out':QUERY_CASA_DEBIT_TRANS
                   , 'casa_trans_in':QUERY_CASA_CREDIT_TRANS
                   , 'eb_no_acct':QUERY_NO_ACCT_OUT
                  }

In [6]:
df_feature = create_feature('df_label_old_a_221206.pkl', dict_df_feature)

  0%|          | 0/4 [00:00<?, ?it/s]

In [12]:
df_feature.drop(columns=['SO_CIF', 'OBS_DATE'], inplace=True)

In [15]:
filename_feature = 'df_feature_old.pkl'
buffer_feature = pickle.dumps(df_feature)
wslib.save_data(filename_feature, buffer_feature, overwrite = True)

{'name': 'df_feature_old.pkl',
 'asset_type': 'data_asset',
 'asset_id': 'e4c0715c-f561-47ba-a36b-1dd4868f5f37',
 'attachment_id': '783e4c66-e06b-4834-97f5-efd05b4e8c05',
 'filepath': 'df_feature_old.pkl',
 'data_size': None,
 'mime': 'application/binary',
 'summary': ['created or overwritten file',
  'created data asset',
  'created attachment']}